In [ ]:
import nltk
nltk.download('omw-1.4')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import re
import pandas as pd
from nltk.corpus import stopwords
from wordcloud import WordCloud, STOPWORDS
import spacy

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Reviews/Reviews1.csv')
df.head()

In [ ]:
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

In [ ]:
!pip install contractions

In [ ]:
!pip install fasttext

In [ ]:
import pandas as pd
import numpy as np
import nltk
import string
import fasttext
import contractions
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer
plt.xticks(rotation=70)
pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', 100)
%matplotlib inline

In [ ]:
for col in df.columns:
    print(col, df[col].isnull().sum())

In [ ]:
df.isnull().sum()

In [ ]:
rws = df.loc[:, ['ProductId','Summary','Text']]

In [ ]:
#Expanding Contradictions
rws['Summary'] = rws['Text'].apply(lambda x: [contractions.fix(word) for word in x.split()])
rws.head()

In [ ]:
rws['review_description_str'] = [' '.join(map(str, l)) for l in rws['Summary']]
rws.head()

In [ ]:
# Define text preprocessing function
def clean(text):
    import html
    import string
    import nltk
    nltk.download('wordnet')

    line = html.unescape(text)
    line = line.replace("can't", 'can not')
    line = line.replace("n't", " not")
    # Pad punctuations with white spaces
    pad_punct = str.maketrans({key: " {0} ".format(key) for key in string.punctuation})
    line = line.translate(pad_punct)
    line = line.lower()
    line = line.split()
    lemmatizer = nltk.WordNetLemmatizer()
    line = [lemmatizer.lemmatize(t) for t in line]

    # Negation handling
    # Add "not_" prefix to words behind "not", or "no" until the end of the sentence
    tokens = []
    negated = False
    for t in line:
        if t in ['not', 'no']:
            negated = not negated
        elif t in string.punctuation or not t.isalpha():
            negated = False
        else:
            tokens.append('not_' + t if negated else t)

    invalidChars = str(string.punctuation.replace("_", ""))
    bi_tokens = list(nltk.bigrams(line))
    bi_tokens = list(map('_'.join, bi_tokens))
    bi_tokens = [i for i in bi_tokens if all(j not in invalidChars for j in i)]
    tri_tokens = list(nltk.trigrams(line))
    tri_tokens = list(map('_'.join, tri_tokens))
    tri_tokens = [i for i in tri_tokens if all(j not in invalidChars for j in i)]
    tokens = tokens + bi_tokens + tri_tokens

    return tokens

In [ ]:
example = clean("I don't liked the food")
print(example)

In [ ]:
!pip install pyspark

In [ ]:
from pyspark.shell import spark
spark.conf.set('spark.sql.shuffle.partitions', '8')

In [ ]:
# importing packages
import pandas as pd

# create data
df1 = pd.DataFrame(df)

# view data
display(df1)

In [ ]:
# Using groupby() and count()
df2 = df1.groupby(['Score'])['Score'].count()
df2.head()

In [ ]:
!pip install -U sentence-transformers

In [ ]:
nltk.download('stopwords')

In [ ]:
import codecs
import csv
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = set(stopwords.words('english'))

In [ ]:
from spacy.lang.en import English

# Load English tokenizer, tagger, parser, NER and word vectors
nlp = English()

text = """He determined to drop his litigation with the monastry, and relinguish his claims to the wood-cuting and
fishery rihgts at once. He was the more ready to do this becuase the rights had become much less valuable, and he had
indeed the vaguest idea where the wood and river in question were."""

#  "nlp" Object is used to create documents with linguistic annotations.
my_doc = nlp(text)

# Create list of word tokens
token_list = []
for token in my_doc:
    token_list.append(token.text)

from spacy.lang.en.stop_words import STOP_WORDS

# Create list of word tokens after removing stopwords
filtered_sentence =[]

for word in token_list:
    lexeme = nlp.vocab[word]
    if lexeme.is_stop == False:
        filtered_sentence.append(word)
print(token_list)
print(filtered_sentence)

In [ ]:
!pip3 install sentence_transformers
#!pip3 install transformers

In [ ]:
!pip install -U spacy
!python -m spacy download en_core_web_lg

In [ ]:
print(df.iloc[123]['Text'])
print(df.iloc[1243]['Text'])
print(df.iloc[23]['Text'])
print(df.iloc[389]['Text'])

In [ ]:
# starts 4-5: Positive(1), stars 1-2: Negative(3), stars 3: Neutral(2)
def map_sentiment(rating):
    if(int(rating)==3):
        return 2
    elif(int(rating)<3):
        return 3
    else:
        return 1

df['Score']
review_sentiments=[map_sentiment(s) for s in df['Score']]
df['sentiments']=review_sentiments

In [ ]:
df.isna().sum()

In [ ]:
df.head()

In [ ]:
df['Date']=pd.to_datetime(df['Date'])
df.value_counts(['ProductId'])

Most Reviewed Brand

In [ ]:
# Get most review business ids
ids=df.value_counts(['ProductId'])
ids=[list(ids.index[i])[0] for i in range(len(ids))]

In [ ]:
data0=df[df['ProductId']==ids[0]]
data1=df[df['ProductId']==ids[1]]
data2=df[df['ProductId']==ids[2]]
data3=df[df['ProductId']==ids[3]]
data4=df[df['ProductId']==ids[4]]
data=pd.concat([data0,data1,data2,data3,data4])
len(data)
data.to_csv('review_business_merge.csv')

In [ ]:
data.head()

In [ ]:
data_most_reviewed_store=df[df['ProductId']=='B007JFMH8M'].sort_values(by=['Time'])
data_most_reviewed_store.shape
business_rating=data_most_reviewed_store.iloc[0]['Score']
print('Overall rating of the store:',business_rating)

In [ ]:
# Import necessary libraries
import pandas as pd
import plotly as py
import cufflinks
import plotly.express as px
import plotly.graph_objects as go
from wordcloud import WordCloud, STOPWORDS
from branca.element import Figure
import folium
import matplotlib.pyplot as pPlot
import numpy as npy
from PIL import Image
from IPython.display import Image as img

In [ ]:
# Visualize the ratings

rating_5=len(df[df['Score']==5])
rating_4=len(df[df['Score']==4])
rating_3=len(df[df['Score']==3])
rating_2=len(df[df['Score']==2])
rating_1=len(df[df['Score']==1])
def getStarRatings():
    fig = go.Figure()
    fig.add_trace(go.Bar(
            y=[1],
            x=[rating_1],
            name='1',
            orientation='h',marker=dict(
        color='rgb(255, 51, 51)'
    )))
    fig.add_trace(go.Bar(
            y=[2],
            name='2',
            x=[rating_2],
            orientation='h',marker=dict(
        color='rgb(255, 92, 51)'
    )))
    fig.add_trace(go.Bar(
            y=[3],
            name='3',
            x=[rating_3],
            orientation='h',marker=dict(
        color='rgb(255, 255, 77)'
    )))
    fig.add_trace(go.Bar(
            y=[4],
            name='4',
            x=[rating_4],
            orientation='h',marker=dict(
        color='rgb(77, 255, 166)'
    )))
    fig.add_trace(go.Bar(
            y=[5],
            name='5',
            x=[rating_5],
            orientation='h',marker=dict(
        color='rgb(166, 255, 77)'

     ))),
    fig.update_layout(
     autosize=False,
     width=500,
     height=500,
     title='Distribution of Review Ratings'
    )
    return fig
getStarRatings()

In [ ]:
print('No of positive Review:',str(len(df[df['sentiments']==1])))
print('No of negative Review:',str(len(df[df['sentiments']==3])))
print('No of neutral Review:',str(len(df[df['sentiments']==2])))
sentimments_dict={3:'Negative',2:'Neutral',1:'Positive'}
sentiment_names=[sentimments_dict[int(i)] for i in data_most_reviewed_store['sentiments'].values]
data_most_reviewed_store['sentiment_name']=sentiment_names
fig = px.pie(data_most_reviewed_store, values='sentiments', names='sentiment_name',color='sentiment_name',color_discrete_map={'Neutral':'yellow','Negative':'cyan','Positive':'green'})
fig.update_layout(
     autosize=False,
     title='Distribution of Review Sentiments'
    )
fig.show()

In [ ]:
date_str=[en.strftime('%Y') for  en in data_most_reviewed_store['Date']]
data_most_reviewed_store['Date_year']=date_str
# Sum the number of reviews per year
data_most_reviewed_store_timeseries_pos=data_most_reviewed_store[data_most_reviewed_store['sentiments']==1].groupby('Date_year')['Date_year','sentiments'].sum()
data_most_reviewed_store_timeseries_neg=data_most_reviewed_store[data_most_reviewed_store['sentiments']==3].groupby('Date_year')['Date_year','sentiments'].sum()
data_most_reviewed_store_timeseries_neu=data_most_reviewed_store[data_most_reviewed_store['sentiments']==2].groupby('Date_year')['Date_year','sentiments'].sum()
data_most_reviewed_store_timeseries_pos['Rating']=['Positive' for i in range(len(data_most_reviewed_store_timeseries_pos))]
data_most_reviewed_store_timeseries_neg['Rating']=['Negative' for i in range(len(data_most_reviewed_store_timeseries_neg))]
data_most_reviewed_store_timeseries_neu['Rating']=['Neutral' for i in range(len(data_most_reviewed_store_timeseries_neu))]
data_most_reviewed_store_timeseries=pd.concat([data_most_reviewed_store_timeseries_pos,data_most_reviewed_store_timeseries_neg])
data_most_reviewed_store_timeseries=pd.concat([data_most_reviewed_store_timeseries,data_most_reviewed_store_timeseries_neu])
# No of reviews per year
data_most_reviewed_store_timeseries['Year']=data_most_reviewed_store_timeseries.index
data_most_reviewed_store_timeseries.head()

In [ ]:
from plotly.subplots import make_subplots
fig = go.Figure()

fig.add_trace(go.Scatter(x=list(data_most_reviewed_store_timeseries.index),
                         y=data_most_reviewed_store_timeseries_pos['sentiments'],
                         mode='lines',
                         name='positive',
                         line=dict(color='rgb(0,245,153)', width=1)))
fig.add_trace(go.Scatter(x=list(data_most_reviewed_store_timeseries.index),
                         y=data_most_reviewed_store_timeseries_neg['sentiments'],
                         mode='lines',
                         name='negative',
                         line=dict(color='rgb(255, 102, 102)', width=1)))
fig.add_trace(go.Scatter(x=list(data_most_reviewed_store_timeseries.index),
                         y=data_most_reviewed_store_timeseries_neu['sentiments'],
                         mode='lines',
                         name='neutral',
                         line=dict(color='rgb(102, 102, 255)', width=1)))
fig.update_layout(
     autosize=False,
     title='Trend of sentiments over time'
    )

In [ ]:
fig = px.bar(data_most_reviewed_store_timeseries.tail(30), x='Year', y='sentiments',color='Rating')
fig.update_layout(
     autosize=False,
     title='Trend of sentiments over time - Bar Graph'
    )
fig.show()

In [ ]:
corpus_pos=" ".join(list(data_most_reviewed_store[data_most_reviewed_store['sentiments']==1]['Text']))
corpus_neg=" ".join(list(data_most_reviewed_store[data_most_reviewed_store['sentiments']==2]['Text']))

In [ ]:
STOPWORDS.add("doughnut")
STOPWORDS.add("donuts")
STOPWORDS.add("voodoo")

In [ ]:
def create_word_cloud(string):
    cloud = WordCloud(background_color = "white", max_words = 15000, stopwords = set(STOPWORDS))
    cloud.generate(string)
    cloud.to_file("wordCloud.png")

# Postitive
create_word_cloud(corpus_pos.lower())
img('/content/wordCloud.png')

In [ ]:
create_word_cloud(corpus_neg.lower())
img('/content/wordCloud.png')

In [ ]:
# Import Necessary Libraries
import re
import nltk
import numpy as np
import itertools
import pickle
import spacy
from tqdm.notebook import tqdm as tqdm
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm_notebook as tqdm
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
from sentence_transformers import SentenceTransformer

In [ ]:
# Stopwards
stopwords = list(set(stopwords.words("english")))
stopwords+=['voodoo','doughnuts','doughnut']

In [ ]:
# Filtering the dataset based on Review Sentiments
positive_reviews=data_most_reviewed_store[data_most_reviewed_store['sentiments']==1]
negative_reviews=data_most_reviewed_store[data_most_reviewed_store['sentiments']==3]
neutral_reviews=data_most_reviewed_store[data_most_reviewed_store['sentiments']==2]

In [ ]:
# Method to preprocess the data
def preprocess(data):
    # Preprocessing Texts
    preprocessed_texts = []
    lem = WordNetLemmatizer()
    # Cleaing the data, removing stopwords
    for sent in tqdm(data):
        sent = sent.replace('\\r', ' ')
        sent = sent.replace('\\"', ' ')
        sent = sent.replace('\\n', ' ')
        sent = re.sub('[^A-Za-z ]+', ' ', sent)
        # lemmatizing
        sent=' '.join(lem.lemmatize(word) for word in sent.split() if word not in stopwords )
        preprocessed_texts.append(sent.lower().strip())
    return preprocessed_texts

preprocessed_texts_neg=preprocess(negative_reviews['Text'].values)
preprocessed_texts_pos=preprocess(positive_reviews['Text'].values)
preprocessed_texts_neu=preprocess(neutral_reviews['Text'].values)

In [ ]:
preprocessed_texts_pos[67]

In [ ]:
# Import Necessary Libraries
import re
import nltk
import numpy as np
import itertools
import spacy
from tqdm.notebook import tqdm as tqdm
import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
from sentence_transformers import SentenceTransformer
from nltk.corpus import stopwords

class CustomerReviewKeywords1:
    # Method to preprocess the data
    stopwords_ = list(set(stopwords.words("english")))
    stopwords_+=['voodoo','doughnuts','doughnut','voodoodoughnut']
    def preprocess(self,data,stopwords):
        # Preprocessing Texts
        preprocessed_texts = []
        lem = WordNetLemmatizer()
        # Cleaing the data, removing stopwords
        for sent in data:
            sent = sent.replace('\\r', ' ')
            sent = sent.replace('\\"', ' ')
            sent = sent.replace('\\n', ' ')
            sent = re.sub('[^A-Za-z ]+', ' ', sent)
            # lemmatizing
            sent=' '.join(word for word in sent.split() if word not in stopwords)
            preprocessed_texts.append(sent.lower().strip())
        return preprocessed_texts

    def max_sum_sim(self,doc_embedding, candidate_embeddings, candidates, top_n, nr_candidates):
        # Calculate distances and extract keywords
        distances = cosine_similarity(doc_embedding, candidate_embeddings)
        distances_candidates = cosine_similarity(candidate_embeddings,
                                            candidate_embeddings)

        # Get top_n words as candidates based on cosine similarity
        words_idx = list(distances.argsort()[0][-nr_candidates:])
        words_vals = [candidates[index] for index in words_idx]
        distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]

        # Calculate the combination of words that are the least similar to each other
        min_sim = np.inf
        candidate = None
        for combination in itertools.combinations(range(len(words_idx)), top_n):
            sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
            if sim < min_sim:
                candidate = combination
                min_sim = sim
        return [words_vals[idx] for idx in candidate]

    # Diversify the keywords using max sum similarity, higher the value of nr_candidates higher the diversity
    def extract_keywords_bert_diverse(self,doc,stopwords,top_n=10,nr_candidates=20):
        n_gram_range = (1,1)
        # Extract candidate words/phrases using count vectorizer (TF-IDF Scores)
        count = CountVectorizer(ngram_range=n_gram_range, stop_words=stopwords).fit([doc])
        candidates = count.get_feature_names()
        # Embeddings of the document using Bert
        model = SentenceTransformer('distilbert-base-nli-mean-tokens')
        doc_embedding = model.encode([doc])
        candidate_embeddings = model.encode(candidates)
        keywords=self.max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n, nr_candidates)
        return keywords

    # Method to get the trending keywords
    def get_trending_keywords(self,data_most_reviewed_store,num_keywords=5):
        # Stopwards
        stopwords_ = list(set(stopwords.words("english")))
        stopwords_+=['voodoo','doughnuts','doughnut','voodoodoughnut','donut','donuts']
        # Filtering the dataset based on Review Sentiments
        positive_reviews=data_most_reviewed_store[data_most_reviewed_store['sentiments']==1]
        negative_reviews=data_most_reviewed_store[data_most_reviewed_store['sentiments']==3]
        neutral_reviews=data_most_reviewed_store[data_most_reviewed_store['sentiments']==2]
        preprocessed_texts_neg=self.preprocess(negative_reviews['Text'].values,stopwords_)
        preprocessed_texts_pos=self.preprocess(positive_reviews['Text'].values,stopwords_)
        preprocessed_texts_neu=self.preprocess(neutral_reviews['Text'].values,stopwords_)
        keywords={}
        corpus=' '.join(preprocessed_texts_pos[-500::])
        keywords['positive']=self.extract_keywords_bert_diverse(corpus,stopwords_,num_keywords)
        corpus=' '.join(preprocessed_texts_neg[-500::])
        keywords['negative']=self.extract_keywords_bert_diverse(corpus,stopwords_,num_keywords)
        return keywords

In [ ]:
ck=CustomerReviewKeywords1()
data_most_reviewed_store=df[df['ProductId']==ids[5]].sort_values(by=['Date'])
kw=ck.get_trending_keywords(data_most_reviewed_store,5)

In [ ]:
df1=pd.DataFrame(columns=['id','pos_keywords','neg_keywords'])
review_ids=[]
pos_keywords=[]
neg_keywords=[]

for i in range(10):
    ck=CustomerReviewKeywords1()
    data_most_reviewed_store=df[df['ProductId']==ids[int(i)]].sort_values(by=['Date'])
    kw=ck.get_trending_keywords(data_most_reviewed_store,num_keywords=5)
    review_ids.append(ids[int(i)])
    pos_keywords.append(kw['positive'])
    neg_keywords.append(kw['negative'])
df1['id']=review_ids
df1['pos_keywords']=pos_keywords
df1['neg_keywords']=neg_keywords
df1.head()

In [ ]:
data_most_reviewed_store.columns

In [ ]:
def recent_reviews(review_business_data_merged,id):
    data_most_reviewed_store=review_business_data_merged[review_business_data_merged['ProductId']==id].sort_values(by=['Date'],ascending=False)
    sentimments_dict={3:'Negative',2:'Neutral',1:'Positive'}
    sentiment_names=[sentimments_dict[int(i)] for i in data_most_reviewed_store['sentiments'].values]
    data_most_reviewed_store['sentiment']=sentiment_names
    data_most_reviewed_store=data_most_reviewed_store.drop(columns=['ProductId','UserId','ProfileName'])
    data_most_reviewed_store=data_most_reviewed_store[['Text','sentiment','Date','Score']]
    data_most_reviewed_store=data_most_reviewed_store.rename(columns={'Score':'Rating','Text':'Review'})
    return data_most_reviewed_store

recent_reviews(df,ids[1])

In [ ]:
import pyspark
spark.conf.set('spark.sql.shuffle.partitions', '8')

In [ ]:
# load .csv data
kindle_csv = spark.read.format("csv").option('header','true').load("/content/drive/MyDrive/Reviews/Reviews1.csv")

In [ ]:
kindle_csv.show()

In [ ]:
#negative (label = 1)  positive (label = 0)
kindle_csv.createOrReplaceTempView('kindle_view')

data_csv = spark.sql('''
  SELECT CASE WHEN Score<4 THEN 1
          ELSE 0
          END as label,
          Text as text
  FROM kindle_view
  WHERE length(text)>2''')

data_csv.groupBy('label').count().show()

In [ ]:
# Sampling data
pos = data_csv.where('label=0').sample(False, 0.05, seed=1220)
neg = data_csv.where('label=1').sample(False, 0.25, seed=1220)
data = pos.union(neg)
data.groupBy('label').count().show()

In [ ]:
# Negative reviews are on average longer than the positive reviews, but not significantly longer
from pyspark.sql.functions import length
data.withColumn('review_length', length('text')).groupBy('label').avg('review_length').show()

In [ ]:
# Perform data preprocessing
from pyspark.sql.functions import udf, col, size
from pyspark.sql.types import ArrayType, StringType
clean_udf = udf(clean, ArrayType(StringType()))
data_tokens = data.withColumn('tokens', clean_udf(col('text')))
data_tokens.show(3)

In [ ]:
# Split data to 70% for training and 30% for testing
training, testing = data_tokens.randomSplit([0.7,0.3], seed=1220)
training.groupBy('label').count().show(3)

In [ ]:
training.cache()

Naive Bayes Model

In [ ]:
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.ml import Pipeline

count_vec = CountVectorizer(inputCol='tokens', outputCol='c_vec', minDF=5.0)
idf = IDF(inputCol="c_vec", outputCol="features")

In [ ]:
# Naive Bayes model
from pyspark.ml.classification import NaiveBayes
nb = NaiveBayes()

pipeline_nb = Pipeline(stages=[count_vec, idf, nb])

model_nb = pipeline_nb.fit(training)
test_nb = model_nb.transform(testing)
test_nb.show()

Naive Bayes Model Using default parameters

In [ ]:
# Naive Bayes model ROC
from pyspark.ml.evaluation import BinaryClassificationEvaluator
roc_nb_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')
roc_nb = roc_nb_eval.evaluate(test_nb)
print("ROC of the NB model: {}".format(roc_nb))

ROC of the NB model: 0.8636859575655796


In [ ]:
# Naive Bayes model accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_nb_eval = MulticlassClassificationEvaluator(metricName='accuracy')
acc_nb = acc_nb_eval.evaluate(test_nb)
print("Accuracy of the NB model: {}".format(acc_nb))

Accuracy of the NB model: 0.862066846891128


Logistic Regressions

In [ ]:
# Logistic Regression model
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
lgr = LogisticRegression(maxIter=5)
pipeline_lgr = Pipeline(stages=[count_vec, idf, lgr])

model_lgr = pipeline_lgr.fit(training)
test_lgr = model_lgr.transform(testing)

In [ ]:
# Logistic Regression model ROC
from pyspark.ml.evaluation import BinaryClassificationEvaluator
roc_lgr_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')
roc_lgr = roc_lgr_eval.evaluate(test_lgr)
print("ROC of the model: {}".format(roc_lgr))

ROC of the model: 0.8801258533761804


In [ ]:
# Logistic Regression model accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_lgr_eval = MulticlassClassificationEvaluator(metricName='accuracy')
acc_lgr = acc_lgr_eval.evaluate(test_lgr)
print("Accuracy of the model: {}".format(acc_lgr))

Accuracy of the model: 0.8812361759646105


Linear SVC Model

In [ ]:
# Linear SVC model
from pyspark.ml.feature import CountVectorizer, IDF
from pyspark.ml import Pipeline
from pyspark.ml.classification import LinearSVC
lsvc = LinearSVC(maxIter=5)
pipeline_lsvc = Pipeline(stages=[count_vec, idf, lsvc])

model_lsvc = pipeline_lsvc.fit(training)
test_lsvc = model_lsvc.transform(testing)

In [ ]:
# Linear SVC model ROC
from pyspark.ml.evaluation import BinaryClassificationEvaluator
roc_lsvc_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='label')
roc_lsvc = roc_lsvc_eval.evaluate(test_lsvc)
print("ROC of the model: {}".format(roc_lsvc))

ROC of the model: 0.874372286344139


In [ ]:
# Linear SVC model accuracy
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
acc_lsvc_eval = MulticlassClassificationEvaluator(metricName='accuracy')
acc_lsvc = acc_lsvc_eval.evaluate(test_lsvc)
print("Accuracy of the model: {}".format(acc_lsvc))

Accuracy of the model: 0.8741705578766281


***Prediction of New Reviews By Model***

Randomly choosen five reviews from the Kindle book The Brave Ones: A Memoir of Hope, Pride and Military Service, by Michael J.MacLeod. The suffixes "_1", "_2", ..., "_5" indicate the real overall review stars 1, 2, ..., 5.



In [ ]:
review_1 = ["WOW!!! No words describe how bland this book is. It took me a lot to even pick up to read. I would definitely not recommend this book."]
review_2 = ["A first person account of the war in Afghanistan. It skipps around a lot and is like a never-ending news article. On the positive side, you do get a feel for what desert fighting is like from a soldiers point of view."]
review_3 = ["I liked the premise and most of the book. At the end parts I lost a little interest because I lost the thread of who was who. War is hell. MacLeod did his service unlike most of us."]
review_4 = ["Very informative first person account of the the daily life of a US Paratrooper. From training to deployment in combat situations in Afghanistan. Well worth the read and makes you really understand and appreciate their sacrifices"]
review_5 = ["This is perhaps the best wrote book I have ever read. Articulate and thought provoking. Not just a riveting account of actual combat, but Michael was able to do what few before him have...captured the essence of what one feels as the battle unfolds. Perhaps most of all, I am grateful to call this author 'Fellow Warrior' Airborne all the way!!!"]

In [ ]:
from pyspark.sql.types import *
schema = StructType([StructField("text", StringType(), True)])

text = [review_1, review_2, review_3, review_4, review_5]
review_new = spark.createDataFrame(text, schema=schema)

In [ ]:
# Data preprocessing
review_new_tokens = review_new.withColumn('tokens', clean_udf(col('text')))
review_new_tokens.show()

+--------------------+--------------------+
|                text|              tokens|
+--------------------+--------------------+
|WOW!!! No words d...|[wow, not_word, n...|
|A first person ac...|[a, first, person...|
|I liked the premi...|[i, liked, the, p...|
|Very informative ...|[very, informativ...|
|This is perhaps t...|[this, is, perhap...|
+--------------------+--------------------+



In [ ]:
# Prediction using Linear SVC Model
result = model_lsvc.transform(review_new_tokens)
result.select('text', 'prediction').show()

+--------------------+----------+
|                text|prediction|
+--------------------+----------+
|WOW!!! No words d...|       1.0|
|A first person ac...|       1.0|
|I liked the premi...|       1.0|
|Very informative ...|       0.0|
|This is perhaps t...|       0.0|
+--------------------+----------+



In [ ]:
# Prediction using Logistic Regression Model
result = model_lgr.transform(review_new_tokens)
result.select('text', 'prediction').show()

+--------------------+----------+
|                text|prediction|
+--------------------+----------+
|WOW!!! No words d...|       1.0|
|A first person ac...|       1.0|
|I liked the premi...|       1.0|
|Very informative ...|       0.0|
|This is perhaps t...|       1.0|
+--------------------+----------+



In [ ]:
# Prediction using Naive Bayes Model
result = model_nb.transform(review_new_tokens)
result.select('text', 'prediction').show()

+--------------------+----------+
|                text|prediction|
+--------------------+----------+
|WOW!!! No words d...|       1.0|
|A first person ac...|       1.0|
|I liked the premi...|       1.0|
|Very informative ...|       0.0|
|This is perhaps t...|       1.0|
+--------------------+----------+

